In [16]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
import os

load_dotenv(override=True)

True

In [17]:

def describe_industry(query):
    # pass
    prompt = ChatPromptTemplate.from_messages([
            ("system", """
        당신은 주식 시장에 대한 통찰력 있는 분석을 제공하는 전문 금융 애널리스트입니다.
        개인 투자자들이 시장 뉴스를 이해하고 현명한 판단을 내릴 수 있도록 돕는 역할을 합니다.

        당신의 목표는 다음과 같습니다:
        - 현재 뉴스가 무엇을 의미하는지 설명하고,
        - 과거 유사 뉴스와 비교해 변화된 점을 분석하며,
        - 관련 산업의 특성을 바탕으로 어떤 영향이 있을 수 있는지 알려줍니다.

        설명은 투자자 입장에서 알기 쉽게 풀어주고, 전문 용어는 피하며, 실제 사례나 비유를 활용해 이해를 돕습니다.
        투자 판단에 참고할 만한 시사점이나 주의할 점도 포함해 주세요.
        """),
            ("human", """
        아래 세 가지 정보를 바탕으로 종합적인 설명을 해주세요.

        ## 현재 뉴스:
        {current_news}

        ## 과거 뉴스:
        {past_news}

        ## 관련 산업 정보:
        {industry_context}

        형식: 2~3개의 문단으로, 명확하고 쉽게 설명해 주세요.  
        분석은 한국어로 작성해 주세요.
        """)
    ])

    llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=4095)

    chain = prompt | llm | StrOutputParser()

    return chain.invoke(query)

In [18]:

from langchain_core.runnables import RunnableLambda


r1 = RunnableLambda(describe_industry)

In [22]:

from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = PineconeVectorStore(
    index_name="lastproject",
    embedding=embedding,
    namespace="industry"
)

query = """
SK Telecom (SKT) is continuing to suffer from a customer exodus, with numbers rising rapidly, especially after it promised to waive contract termination fees for everyone in response to a recent cybersecurity incident.

A total of 17,376 SKT subscribers switched to rival carriers on Thursday, according to the telecom industry on Friday. Of these, 8,915 moved to KT and 8,461 to LG U+.

In contrast, only 10,720 subscribers switched to SKT from KT and LG U+, resulting in a net loss of 6,656 subscribers in just one day.

Since Saturday, when SKT began waiving early termination fees, the company has averaged over 10,000 daily subscriber losses, with the exception of Sunday. The cumulative number of subscribers who left SKT between Saturday to Thursday has reached 75,214, with a net outflow of 28,566.
관련 산업이 어떤 산업인지 설명해주고, 왜 이 기사와 관련됐는지 구체적으로 설명해줘
"""

# vector_store.similarity_search(query, namespace=os.getenv("industry"), k=3)

results = vector_store.similarity_search(query, namespace="industry", k=3)

past_news = "과거 LG U+의 개인정보 유출 사건 당시에도 유사한 해지 사태가 있었습니다."

for i, doc in enumerate(results):
    industry_name = doc.page_content.split("\n")[0].replace("KRX 업종명:", "").strip()
    industry_text = doc.page_content

    print(f"\n[{i+1}] 📌 유사 산업: {industry_name}")

    explanation = describe_industry({
        "current_news": query,
        "past_news": past_news,
        "industry_context": industry_text
    })

    print(f"📍 관련 이유:\n{explanation}")
    print("---")


[1] 📌 유사 산업: ﻿ 일반서비스
📍 관련 이유:
현재 SK텔레콤(SKT)의 고객 이탈 사태는 최근 발생한 사이버 보안 사고에 대한 대응으로, 계약 해지 수수료를 면제하면서 가속화되고 있습니다. 이는 고객들이 보안에 대한 신뢰를 잃고 경쟁사로 이동하는 결과를 초래하고 있습니다. 하루에만 6,656명의 순손실을 기록했으며, 일주일 동안 28,566명의 순손실을 보였습니다. 이는 과거 LG U+의 개인정보 유출 사건 당시에도 유사한 해지 사태가 있었던 것과 유사합니다. 당시에도 고객들은 보안 문제로 인해 불안감을 느끼고 다른 통신사로 이동하는 경향을 보였습니다.

관련 산업은 일반서비스 업종으로, 콜센터 및 텔레마케팅 서비스업이 포함됩니다. 이 산업은 고객의 불만이나 요청을 처리하고, 상품 및 서비스를 홍보하는 역할을 합니다. SKT의 경우, 고객 서비스와 관련된 부분에서 신뢰를 잃게 되면, 고객 이탈이 가속화될 수 있습니다. 특히, 고객의 불만을 효과적으로 처리하지 못하거나, 보안 문제에 대한 적절한 대응이 부족할 경우, 고객의 신뢰를 회복하기 어려울 수 있습니다.

투자자 입장에서 주의할 점은, 이러한 고객 이탈이 장기적으로 SKT의 수익성에 부정적인 영향을 미칠 수 있다는 것입니다. 고객 기반이 줄어들면 매출 감소로 이어질 수 있으며, 이는 주가에도 부정적인 영향을 미칠 수 있습니다. 따라서, SKT가 보안 문제를 어떻게 해결하고 고객 신뢰를 회복할 수 있을지 주의 깊게 지켜볼 필요가 있습니다. 또한, 경쟁사인 KT와 LG U+가 이러한 상황을 어떻게 활용하고 있는지도 관찰할 필요가 있습니다.
---

[2] 📌 유사 산업: ﻿ 일반서비스
📍 관련 이유:
현재 SK텔레콤(SKT)의 고객 이탈 사태는 최근 발생한 사이버 보안 사고에 대한 대응으로, 계약 해지 수수료를 면제하면서 가속화되고 있습니다. 이는 고객들이 보안에 대한 신뢰를 잃고 경쟁사로 이동하는 결과를 초래하고 있습니다. 하루에만 6,656명의 순손실을 기록하며, 일주일 동안 28,566명의 순